In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import folium
import requests
import json

import pandas as pd
import numpy as np

## pip install openlostcat
from openlostcat.main_osm_categorizer import MainOsmCategorizer
from ipywidgets import interact

In [2]:
budapest_req = """
[out:json];
    (area["name"="Budapest"];) -> .searchArea;
    node[tourism=hotel](area.searchArea);
out body;
"""

In [3]:
# pip install openlostcat
from openlostcat.osmqueryutils.ask_osm import ask_osm

budapest_hotels = ask_osm(budapest_req)
len(budapest_hotels['elements'])

140

In [4]:
bp_hotels = pd.DataFrame([[node['id'], node['lat'], node['lon'], node['tags'].get('name', 'NoName'),  
                           node['tags']] for node in budapest_hotels['elements']], 
                         columns = ['id','lat','lng', 'name', 'tags'])

In [5]:
from openlostcat.osmqueryutils.ask_osm import ask_osm_around_point_df

bp_hotels["osm"] = bp_hotels[["lat", "lng"]].T.apply(lambda x: ask_osm_around_point_df(x, distance = 300))

In [6]:
len(bp_hotels.loc[bp_hotels.osm.isna(), "osm"])

0

In [7]:
if len(bp_hotels.loc[bp_hotels.osm.isna(), "osm"]) > 0:
    bp_hotels.loc[bp_hotels.osm.isna(), "osm"] = \
    bp_hotels[bp_hotels.osm.isna()][["lat", "lng"]].T.apply(lambda x: ask_osm_around_point_df(x, distance = 300))

In [8]:
len(bp_hotels.loc[bp_hotels.osm.isna(), "osm"])

0

In [56]:
import folium

def show_geo(related_geo, color_map, get_color_func):
    tmp = related_geo
    # extract coordinates
    coords = np.array(list(zip(tmp["lat"],tmp["lng"])))
    # extract other resources
#     names = list(tmp["name"])
#     tags = list(tmp["tags"])
#     Some coding issue: https://github.com/python-visualization/folium/issues/1320
    names = list([str(name.encode('raw_unicode_escape'))[2:-1] for name in tmp.name])
    tags = list([str(str(tag).encode('raw_unicode_escape'))[2:-1] for tag in tmp.tags])
    colors = get_color_func(tmp, color_map)
    m = folium.Map(
        location=coords.mean(axis=0),
        zoom_start=12,
        tiles='Stamen Terrain'
    )
    for i, loc in enumerate(coords):
        folium.Marker(loc, popup='<i>%s</i>' % tags[i], tooltip=str(names[i]), icon=folium.Icon(color=colors[i])).add_to(m)
    display(m)

## Public transport

In [9]:
#from openlostcat.main_osm_categorizer import MainOsmCategorizer

categorizer = MainOsmCategorizer('rules/publictransport_rules.json')
print(categorizer.get_categories_enumerated_key_map())
print(categorizer)

bp_hotels["pt_cat"] = [i[0] for i in bp_hotels.osm.map(categorizer.categorize)]

{0: 'pt_primary_accessible', 1: 'pt_accessible', 2: 'pt_nonaccessible'}
CategoryCatalog:
category rule collection: [
    Category name: pt_primary_accessible
    rules: [
        ANY(
            and(
                {public_transport : {'stop_position'}}, is_optional_key = False
                or[
                    {light_rail : {'yes'}}, is_optional_key = False
                    {subway : {'yes'}}, is_optional_key = False
                    {train : {'yes'}}, is_optional_key = False
                ]
            )
        )
    ]
    Category name: pt_accessible
    rules: [
        OR[
            ANY(
                {public_transport : {'stop_position'}}, is_optional_key = False
            )
            ANY(
                {public_transport : {'platform'}}, is_optional_key = False
            )
            ANY(
                {amenity : {'ferry_terminal'}}, is_optional_key = False
            )
        ]
    ]
    Category name: pt_nonaccessible
    rules: [
        CONST

In [60]:
bp_hotels.pt_cat.value_counts()

1    81
0    54
2     5
Name: pt_cat, dtype: int64

In [59]:
color_map = dict(zip([0, 1, 2], ["green", "orange", "red"]))
get_color_func = lambda df, color_map: list(df["pt_cat"].apply(lambda x: color_map.get(x, "black")))
key_map = dict(zip(['pt_primary_accessible', 'pt_accessible', 'pt_nonaccessible'], [0, 1, 2]))


# show_geo(bp_hotels, color_map, get_color_func)


transport = ['All', 'pt_primary_accessible', 'pt_accessible', 'pt_nonaccessible']
@interact(pt_cat=transport)
def get_transport(pt_cat):
    selected = bp_hotels[bp_hotels.pt_cat == key_map[pt_cat]] if pt_cat != 'All' else bp_hotels
    show_geo(selected, color_map, get_color_func)

interactive(children=(Dropdown(description='pt_cat', options=('All', 'pt_primary_accessible', 'pt_accessible',…

## Nearby

In [13]:
# from openlostcat.main_osm_categorizer import MainOsmCategorizer

categorizer2 = MainOsmCategorizer('rules/nearby.json')
print(categorizer2.get_categories_enumerated_key_map())
print(categorizer2)

nb_cat = [[cat[0] for cat in cat_list] for cat_list in bp_hotels.osm.map(categorizer2.categorize)]
bp_hotels["water_nearby"] = [0 in cats  for cats in nb_cat]
bp_hotels["park_nearby"] = [1 in cats  for cats in nb_cat]

{0: 'water_nearby', 1: 'park_nearby'}
CategoryCatalog:
category rule collection: [
    Category name: water_nearby
    rules: [
        ANY(
            {waterway : {'river'}}, is_optional_key = False
        )
    ]
    Category name: park_nearby
    rules: [
        ANY(
            {leisure : {'park'}}, is_optional_key = False
        )
    ]
]


In [70]:
print("park and water: " + str(bp_hotels[bp_hotels.water_nearby & bp_hotels.park_nearby].shape[0]))
print("only water: " + str(bp_hotels[(bp_hotels.water_nearby) & (bp_hotels.park_nearby == False)].shape[0]))
print("only park: " + str(bp_hotels[(bp_hotels.water_nearby == False) & (bp_hotels.park_nearby)].shape[0]))
print("nothing: " + str(bp_hotels[(bp_hotels.water_nearby == False) & (bp_hotels.park_nearby == False)].shape[0]))

park and water: 9
only water: 4
only park: 105
nothing: 22


In [55]:
color_map = {
    (True, False): "blue",
    (False, True): "green",
    (True, True): "purple",
    (False, False): "black"
}
get_color_func = lambda df, color_map: list(map(lambda x: color_map.get(x, "black"), list(zip(df.water_nearby, df.park_nearby))))
nearby = ['All', 'water_nearby', 'park_nearby', 'both', 'none']
@interact(nearby_cat=nearby)
def get_nerby(nearby_cat):
    water_nearby = lambda x: x[x.water_nearby]
    park_nearby = lambda x: x[x.park_nearby]
    both = lambda x: x[x.water_nearby & x.park_nearby]
    none = lambda x: x[(x.water_nearby == False) & (x.park_nearby == False)]
    switch = {
        'water_nearby': water_nearby,
        'park_nearby': park_nearby,
        'both': both,
        'none': none
    }
    selected = switch.get(nearby_cat, lambda x: x)(bp_hotels)
    show_geo(selected, color_map, get_color_func)

interactive(children=(Dropdown(description='nearby_cat', options=('All', 'water_nearby', 'park_nearby', 'both'…

## Preferred: Mix

In [34]:
# from openlostcat.main_osm_categorizer import MainOsmCategorizer

categorizer3 = MainOsmCategorizer('rules/mix.json')
print(categorizer3.get_categories_enumerated_key_map())
print(categorizer3)

bp_hotels["preferred_cat"] = [i[0] for i in bp_hotels.osm.map(categorizer3.categorize)]

{0: '1st_preferred_location', 1: '2nd_preferred_location', 2: '3rd_preferred_location', 3: 'not_preferred_location'}
CategoryCatalog:
category rule collection: [
    Category name: 1st_preferred_location
    rules: [
        AND(
            REF ##pt_primary_accessible(
                ANY(
                    and(
                        {public_transport : {'stop_position'}}, is_optional_key = False
                        or[
                            {light_rail : {'yes'}}, is_optional_key = False
                            {subway : {'yes'}}, is_optional_key = False
                            {train : {'yes'}}, is_optional_key = False
                        ]
                    )
                )
            )
            REF ##water_AND_park(
                AND(
                    REF ##water_nearby(
                        ANY(
                            {waterway : {'river'}}, is_optional_key = False
                        )
                    )
                    

In [35]:
bp_hotels.preferred_cat.value_counts()

2    60
1    52
3    25
0     3
Name: preferred_cat, dtype: int64

In [54]:
color_map = dict(zip([0, 1, 2, 3], ["green", "blue", "orange", "black"]))
get_color_func = lambda df, color_map: list(df["preferred_cat"].apply(lambda x: color_map.get(x, "black")))
key_map = dict(zip(['1st_preferred_location', '2nd_preferred_location', '3rd_preferred_location', 'not_preferred_location'], [0, 1, 2, 3]))

preffered = ['All', '1st_preferred_location', '2nd_preferred_location', '3rd_preferred_location', 'not_preferred_location']
@interact(preffered_cat=preffered)
def get_nerby(preffered_cat):
    selected = bp_hotels[bp_hotels.preferred_cat == key_map[preffered_cat]] if preffered_cat != 'All' else bp_hotels
    show_geo(selected, color_map, get_color_func)

interactive(children=(Dropdown(description='preffered_cat', options=('All', '1st_preferred_location', '2nd_pre…